# Random Forest Classifer with pipeline and Hyperparameter Tunning.

In [1]:
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [5]:
df.time.unique()

array([0, 1])

In [6]:
# Independent and Dependent Feature. 
X = df.drop(labels=['time'],axis=1) 
y = df.time

In [7]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [8]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [9]:
# Train test split 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## Handle Missing Value
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## Categorical to numerical
from sklearn.compose import ColumnTransformer

In [12]:
categorial_cols = ['sex','smoker','day']
numerical_cols =['total_bill','tip','size']

In [13]:
## Feature Engineering Automation
# Numerical Piplelining 
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
         ('scaler',StandardScaler())
               
    ]
)
# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoder',OneHotEncoder())
    ]
)

In [14]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorial_cols)
])

In [15]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [37]:
## Model Traning Automation 
models= {
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(), 
    'Decision Tree':DecisionTreeClassifier()
}

In [17]:
from sklearn.metrics import accuracy_score

In [39]:
def evaluate_model(X_train,y_train,X_test,y_test,models):

    report = {}
    for i in range(len(models)): 
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)

        # Predict testing data 
        y_test_pred = model.predict(X_test)

        # Get accuracy score for the model
        test_model_score = accuracy_score(y_test,y_test_pred) 

        report[list(models.keys())[i]] = test_model_score

    return report

In [40]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [41]:
classifier = RandomForestClassifier()

In [42]:
# Hyperparameter Tuning
params = {
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [43]:
from sklearn.model_selection import RandomizedSearchCV

In [46]:
cv = RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',verbose=3,cv = 5)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=1.000 total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [47]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 3, 'criterion': 'entropy'}

In [48]:
cv.predict(X_test)

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])